# Azure Kusto
In this notebook we will access Azure Kusto database

In [ ]:
# install all the azure kusto modules
%pip install azure.kusto.data
%pip install azure-kusto-ingest
%pip install azure.identity
%pip install pandas

In [17]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder, ClientRequestProperties
from azure.identity import AzureCliCredential, ChainedTokenCredential, DefaultAzureCredential, ManagedIdentityCredential
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
import pandas as pd
import json

class KustoConnection():
    def __init__(self, configuration : dict):
        uri = configuration.get('cluster_uri')
        if (not str(uri).startswith('https://')):
            uri = "https://" + uri
        if (not str(uri).endswith('.kusto.windows.net')) :
            uri = uri + '.kusto.windows.net'
        self.cluster_uri = uri
        self.database = configuration.get('database')
        cred = ChainedTokenCredential(ManagedIdentityCredential(), AzureCliCredential(), DefaultAzureCredential())
        connection_string = KustoConnectionStringBuilder.with_aad_application_token_authentication(self.cluster_uri, 
                                                                                                   cred.get_token(self.cluster_uri + '/.default').token)        
        # Create a KustoClient instance
        self.client = KustoClient(connection_string)

    def query(self, query: str, properties: ClientRequestProperties = None) -> pd.DataFrame:
        """
        Execute a Kusto query.
        @param query: The query.
        @returns: A Pandas DataFrame.
        """
        response = self.client.execute_query(self.database, query, properties)
        df = dataframe_from_result_table(response.primary_results[0])
        return df

    def get_schema(self, table_name: str) -> dict:
        """
        Get a table schema
        @param table_name: name of table
        @returns a dictionary mapping column names to kusto types
        """
        schema = self.client.execute_mgmt(self.database, f".show table {table_name} schema as json").primary_results[0][0]['Schema']
        return json.loads(schema)

    def close(self):
        """
        Close the Kusto client.
        """
        self.client.close()


kusto_client = KustoConnection({'cluster_uri' : "dpxedwhprod.southcentralus", 'database': "Dpxedwhprod"});
query = "DCEPAllMilestonesCurrent"
response = kusto_client.query(query);
result = response;
print(result);


                    PORVersion SnapshotVersion  ColoTrancheId      Region  \
0    2024-02-02 00:00:00+00:00          Latest          16927        EMEA   
1    2024-02-02 00:00:00+00:00          Latest          17112        EMEA   
2    2024-02-02 00:00:00+00:00          Latest           4513        EMEA   
3    2024-02-02 00:00:00+00:00          Latest          15811        EMEA   
4    2024-02-02 00:00:00+00:00          Latest          18804        EMEA   
...                        ...             ...            ...         ...   
6154 2024-02-02 00:00:00+00:00          Latest          12781        EMEA   
6155 2024-02-02 00:00:00+00:00          Latest          12285        EMEA   
6156 2024-02-02 00:00:00+00:00          Latest          26300  AMERS EAST   
6157 2024-02-02 00:00:00+00:00          Latest          26299  AMERS EAST   
6158 2024-02-02 00:00:00+00:00          Latest          18405        EMEA   

          SubRegion               Country                Campus       Metro